In [ ]:
import os
import pandas as pd
import statsmodels.formula.api as sm

In [ ]:
import matplotlib.pyplot as plt

## Data preparation

In [ ]:
# Import data

Geo_PUF_sheets = pd.read_excel('Geo_variation_PUF.xlsx', sheet_name=None, header=1, usecols="A:B,D:E,G:P, U") # takes 90 seconds
sheets = ['State_county ' + str(i) for i in range (2007, 2019) ]

column_names=['State', 'County', 'Total_pop', 'FFS_pop', 'MA_penetration', 'Average_age',
       'Percent_Female', 'Percent Male', 'Percent_white',
       'Percent_Black', 'Percent_Hispanic',
       'Percent Other/Unknown', 'Percent_duals', 'Risk_score', 'Actual_cost']
data_sets = [Geo_PUF_sheets['State_county ' + str(i)] for i in range (2007, 2019)]

for i in range(0,12):
    data_sets[i].columns=column_names
    data_sets[i].insert(0, 'Year', i+2007)
    
df_geo_var = pd.concat(data_sets, sort=False)

In [ ]:
# Save checkpoint
df_geo_var.to_csv('Geo_var.csv', sep=';', index=False)

In [ ]:
# Load checkpoint
df_geo_var=pd.read_csv('Geo_var.csv', sep=';', dtype='string')

In [ ]:
#Calculate expected costs

num_cols=['Total_pop', 'FFS_pop', 'Average_age','Risk_score', 'Actual_cost']
df_geo_var[num_cols] = df_geo_var[num_cols].apply(pd.to_numeric, errors='coerce')


USPCC=df_geo_var[df_geo_var.County=='NATIONAL TOTAL'][['Year', 'Actual_cost']]
USPCC.columns=['Year', 'USPCC']
df_geo_var=df_geo_var.merge(USPCC, how='inner', on='Year')
df_geo_var['Expected_cost']=df_geo_var.USPCC*df_geo_var.Risk_score

df_geo_var=df_geo_var[~df_geo_var.County.str.contains('TOTAL')]
df_geo_var['OE_cost']= df_geo_var.Actual_cost/df_geo_var.Expected_cost

In [ ]:
# Clean data
def unpercent(series):
    return pd.to_numeric(series.str.slice(start=0, stop=-2), errors='coerce')/100
    

percent_cols=[ 'MA_penetration', 'Percent_Female', 'Percent Male', 'Percent_white',
       'Percent_Black', 'Percent_Hispanic',
       'Percent Other/Unknown', 'Percent_duals']
df_geo_var[percent_cols] = df_geo_var[percent_cols].apply(unpercent)


df_geo_var['FIPS']=df_geo_var.State.str.cat(df_geo_var.County, sep='_')

In [ ]:
# Take out counties with incomplete data

df=df_geo_var.dropna()
kill_list=df.groupby(['FIPS']).count()[df.groupby(['FIPS']).count().Year<12].reset_index().FIPS
df=df[~df.FIPS.isin(kill_list)]

In [ ]:
# Create time lagged variablee

df=df.sort_values(['FIPS', 'Year'], ascending=True).reset_index(drop=True)
df['MA_penetration_lagged_1']=df.groupby(['FIPS'])['MA_penetration'].shift(1)
df=df[df.Year !='2007']

## Analysis

In [ ]:
# Regression

model=sm.ols(formula= 'OE_cost ~ MA_penetration_lagged_1 + Average_age + Percent_Female + Percent_white + Percent_Black  + Percent_Hispanic + Percent_duals + C(Year) + C(FIPS)', data=df)
results = model.fit()
print(results.summary())
print(results.summary().as_latex())

In [ ]:
# Construct chart 1

quantiles=10 #select desired granularity of chart

df_graph=df

plt.plot(range(1,quantiles +1) , df_graph.groupby(pd.qcut(df_graph.MA_penetration_lagged_1, quantiles))['Actual_cost', 'Expected_cost' ].mean())
plt.xlabel('Prior year MA-penetration decile')
plt.ylabel('Average annual FFS cost, $')
plt.title('Chart 1: Actual and expected FFS costs by MA-penetration')
plt.legend(["Actual cost", "Expected cost"])
plt.savefig('Chart 1.pdf')